### Настройки

Начинаем с импорта библиотек

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
DBNAME = 
USER = 
PASSWORD = 
HOST = 
PORT = 

Создаем соединение с заданными параметрами

In [4]:
uri = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}"

In [5]:
engine = create_engine(uri)
pd.set_option('display.min_rows', 100) #увеличим количество отображаемых строк для наглядности

Запрос

3.1 Сколько вакансий есть в базе?

In [6]:
query = f'''SELECT count(*) 
            FROM vacancies
         '''
df = pd.read_sql_query(query, engine)
df

,count
0,49197


3.2 Сколько работодателей?

In [7]:
query = f'''SELECT count(*) 
            FROM employers
         '''
df = pd.read_sql_query(query, engine)
df

,count
0,23501


3.3 Сколько регионов?

In [8]:
query = f'''SELECT count(*) 
            FROM areas
         '''
df = pd.read_sql_query(query, engine)
df

,count
0,1362


3.4 Сколько сфер деятельности?

In [9]:
query = f'''SELECT count(*) 
            FROM industries
         '''
df = pd.read_sql_query(query, engine)
df

,count
0,294


Вывод:
Информация из этого блока сама по себе не представляет интереса, ее можно рассматривать в сравнении с вакансиями из других источников, но сами по себе это просто цифры.

4.1 Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).

In [10]:
query = f'''SELECT a.name AS area, COUNT(v.id) AS cnt
            FROM areas a
            JOIN vacancies v ON a.id = v.area_id
            GROUP BY a.name
            ORDER BY cnt DESC
         '''
df = pd.read_sql_query(query, engine)
df


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


4.2 У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [11]:
query = f'''SELECT COUNT(*) AS cnt
            FROM vacancies
            WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
         '''
df = pd.read_sql_query(query, engine)
df

,cnt
0,24073


4.3 Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.

In [12]:
query = f'''SELECT ROUND(AVG(salary_from)) AS avg_salary_from, ROUND(AVG(salary_to)) AS avg_salary_to
            FROM vacancies
            WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
         '''
df = pd.read_sql_query(query, engine)
df

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4.4 Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.

In [13]:
query = f'''SELECT v.schedule, v.employment, COUNT(*) AS cnt
            FROM vacancies v
            GROUP BY v.schedule, v.employment
            ORDER BY cnt DESC
         '''
df = pd.read_sql_query(query, engine)
df

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


4.5 Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [14]:
query = f'''SELECT v.experience, COUNT(*) AS cnt
            FROM vacancies v
            GROUP BY v.experience
            ORDER BY cnt ASC
         '''
df = pd.read_sql_query(query, engine)
df

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


Вывод:
В базе присутствует много вакансий с низким стажем, а работодатели предпочитают полную занятость. Посчитана средняя минимальная и максимальная зарплата. Но в таблице нет разделения на страны, поэтому такое исследование не будет объективным. Большинство вакансий находится в городах миллионниках и для исследования можно взять только их данные (что тоже не очень хорошо, так как в крупных городах зарплаты выше), что и будет сделано в следующем блоке.

5.1 Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [15]:
query = f'''SELECT e.name, COUNT(*) AS cnt
            FROM employers e
            JOIN vacancies v ON e.id = v.employer_id
            GROUP BY e.name
            ORDER BY cnt DESC
            LIMIT 5
         '''
df = pd.read_sql_query(query, engine)
df

,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


5.2 Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.

Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.

In [16]:
query = f'''SELECT a.name AS area, COUNT(DISTINCT e.id) AS employers
            FROM areas a
            LEFT JOIN employers e ON a.id = e.area
            LEFT JOIN vacancies v ON a.id = v.area_id
            WHERE v.id IS NULL
            GROUP BY a.name
            ORDER BY employers DESC
            LIMIT 1
         '''
df = pd.read_sql_query(query, engine)
df

,area,employers
0,Россия,410


5.3 Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.

In [17]:
query = f'''SELECT e.name AS employer, COUNT(DISTINCT a.id) AS areas
            FROM employers e
            JOIN vacancies v ON e.id = v.employer_id
            JOIN areas a ON v.area_id = a.id
            GROUP BY e.name
            ORDER BY areas DESC
         '''
df = pd.read_sql_query(query, engine)
df

,employer,areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


5.4 Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [18]:
query = f'''SELECT COUNT(*) AS cnt
            FROM employers e
            LEFT JOIN employers_industries ei ON e.id = ei.employer_id
            WHERE ei.industry_id IS NULL
         '''
df = pd.read_sql_query(query, engine)
df

,cnt
0,8419


5.5 Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [19]:
query = f'''SELECT e.name
            FROM employers e
            JOIN employers_industries ei ON e.id = ei.employer_id
            GROUP BY e.name
            HAVING COUNT(ei.industry_id) = 4
            ORDER BY e.name ASC
            LIMIT 1 OFFSET 2
         '''
df = pd.read_sql_query(query, engine)
df

,name
0,2ГИС


5.6 С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [20]:
query = f'''SELECT COUNT(DISTINCT ei.employer_id) AS cnt
            FROM employers_industries ei
            JOIN industries i ON ei.industry_id = i.id
            WHERE i.name = 'Разработка программного обеспечения'
         '''
df = pd.read_sql_query(query, engine)
df

,cnt
0,3553


5.7 Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании, собранных в этой таблице.

In [21]:
query = f'''WITH filtered_vacancies AS (
                SELECT a.name AS area, v.id
                FROM areas a
                JOIN vacancies v ON a.id = v.area_id
                JOIN employers e ON v.employer_id = e.id
                WHERE e.name = 'Яндекс' AND a.name IN (
                    'Москва', 
                    'Санкт-Петербург', 
                    'Новосибирск', 
                    'Екатеринбург', 
                    'Казань', 
                    'Нижний Новгород', 
                    'Красноярск', 
                    'Челябинск', 
                    'Самара', 
                    'Уфа', 
                    'Ростов-на-Дону', 
                    'Краснодар', 
                    'Омск', 
                    'Воронеж', 
                    'Пермь', 
                    'Волгоград')
            )
            SELECT area, COUNT(id) AS vacancies
            FROM filtered_vacancies
            GROUP BY area
            UNION ALL
            SELECT 'Total' AS area, COUNT(id) AS vacancies
            FROM filtered_vacancies
         '''
df = pd.read_sql_query(query, engine)
df

,area,vacancies
0,Казань,25
1,Нижний Новгород,36
2,Екатеринбург,39
3,Новосибирск,35
4,Красноярск,23
5,Омск,21
6,Воронеж,32
7,Пермь,25
8,Уфа,26
9,Челябинск,22


Вывод: Яндекс является самой большой компанией в этой базе - он имеет больше всего вакансий и представлен почти в двух сотнях регионов. А вот средняя зарплата у Яндекса так себе

In [22]:
query = f'''SELECT ROUND(AVG(salary_from)) AS avg_salary_from, ROUND(AVG(salary_to)) AS avg_salary_to
            FROM vacancies v
            JOIN employers e ON v.employer_id = e.id
            WHERE e.name = 'Яндекс' and (salary_from IS NOT NULL OR salary_to IS NOT NULL)
         '''
df = pd.read_sql_query(query, engine)
df

,avg_salary_from,avg_salary_to
0,22467.0,55961.0


В данных помимо городов присутствуют страны и области. Но вакансии связаны только с городами. Возможно, такая структура данных была выбрана для удобства сортировки и фильтрации. Например, если компания имеет офисы в разных городах, областях или странах, то такая структура позволяет легко отслеживать их расположение.

6.1 Сколько вакансий имеет отношение к данным?

In [23]:
query = f'''SELECT COUNT(*) AS cnt
            FROM vacancies
            WHERE name ILIKE '%%data%%' OR name ILIKE '%%данн%%'
         '''
df = pd.read_sql_query(query, engine)
df

,cnt
0,1771


6.2 Сколько есть подходящих вакансий для начинающего дата-сайентиста?

In [24]:
query = f'''SELECT COUNT(*) AS cnt
            FROM vacancies
            WHERE (
               name ILIKE '%%data scientist%%' 
               OR name ILIKE '%%data science%%' 
               OR name ILIKE '%%исследователь данных%%' 
               OR (name ILIKE '%%ml%%' AND name NOT ILIKE '%%html%%') 
               OR name ILIKE '%%machine learning%%' 
               OR name ILIKE '%%машинн%%обучен%%')
            AND (name ILIKE '%%junior%%' OR experience = 'Нет опыта' OR employment = 'Стажировка')
         '''
df = pd.read_sql_query(query, engine)
df

,cnt
0,51


6.3 Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

In [25]:
query = f'''SELECT COUNT(*) AS cnt
            FROM vacancies
            WHERE (
               name ILIKE '%%data scientist%%' 
               OR name ILIKE '%%data science%%' 
               OR name ILIKE '%%исследователь данных%%' 
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%') 
               OR name ILIKE '%%machine learning%%' 
               OR name ILIKE '%%машинн%%обучен%%')
            AND (key_skills ILIKE '%%sql%%' OR key_skills ILIKE '%%postgres%%')
         '''
df = pd.read_sql_query(query, engine)
df

,cnt
0,201


6.4 С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS.

In [26]:
query = f'''SELECT COUNT(*) AS cnt
            FROM vacancies
            WHERE (
               name ILIKE '%%data scientist%%' 
               OR name ILIKE '%%data science%%' 
               OR name ILIKE '%%исследователь данных%%' 
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%') 
               OR name ILIKE '%%machine learning%%' 
               OR name ILIKE '%%машинн%%обучен%%')
            AND key_skills ILIKE '%%python%%'
         '''
df = pd.read_sql_query(query, engine)
df

,cnt
0,351


6.5 Сколько ключевых навыков в среднем указывают в вакансиях для DS?

In [27]:
query = f'''SELECT AVG((LENGTH(key_skills) - LENGTH(REPLACE(key_skills, '\t', ''))) / LENGTH('\t') + 1)
            FROM vacancies
            WHERE (
               name ILIKE '%%data scientist%%' 
               OR name ILIKE '%%data science%%' 
               OR name ILIKE '%%исследователь данных%%' 
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%') 
               OR name ILIKE '%%machine learning%%' 
               OR name ILIKE '%%машинн%%обучен%%')
         '''
df = pd.read_sql_query(query, engine)
df

,avg
0,6.406032


6.6 Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).

In [28]:
query = f'''SELECT experience, ROUND(AVG(
    CASE
        WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL THEN (salary_from + salary_to) / 2
        WHEN salary_from IS NOT NULL THEN salary_from
        WHEN salary_to IS NOT NULL THEN salary_to
    END)) AS avg_salary
            FROM vacancies
            WHERE (name ILIKE '%%data scientist%%' 
                OR name ILIKE '%%data science%%' 
                OR name ILIKE '%%исследователь данных%%' 
                OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%') 
                OR name ILIKE '%%machine learning%%' 
                OR name ILIKE '%%машинн%%обучен%%')
                AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
            GROUP BY experience
         '''
df = pd.read_sql_query(query, engine)
df

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


Вывод: вакансий для джунов мало относительно общего количества вакансий и питон более востребован, чем sql. Зарплата хорошая. У вакансий с питоном средняя зарплата на 10-15 процентов выше, чем у вакансий с sql.

In [29]:
query = f'''SELECT experience, ROUND(AVG(
    CASE
        WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL THEN (salary_from + salary_to) / 2
        WHEN salary_from IS NOT NULL THEN salary_from
        WHEN salary_to IS NOT NULL THEN salary_to
    END)) AS avg_salary
            FROM vacancies
            WHERE (name ILIKE '%%data scientist%%' 
                OR name ILIKE '%%data science%%' 
                OR name ILIKE '%%исследователь данных%%' 
                OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%') 
                OR name ILIKE '%%machine learning%%' 
                OR name ILIKE '%%машинн%%обучен%%')
                AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
                AND key_skills ILIKE '%%python%%'
                AND (key_skills ILIKE '%%sql%%' OR key_skills ILIKE '%%postgres%%')
            GROUP BY experience
         '''
df = pd.read_sql_query(query, engine)
df

,experience,avg_salary
0,Нет опыта,61900.0
1,От 1 года до 3 лет,137000.0
2,От 3 до 6 лет,219500.0


In [30]:
query = f'''SELECT experience, ROUND(AVG(
    CASE
        WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL THEN (salary_from + salary_to) / 2
        WHEN salary_from IS NOT NULL THEN salary_from
        WHEN salary_to IS NOT NULL THEN salary_to
    END)) AS avg_salary
            FROM vacancies
            WHERE (name ILIKE '%%data scientist%%' 
                OR name ILIKE '%%data science%%' 
                OR name ILIKE '%%исследователь данных%%' 
                OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%') 
                OR name ILIKE '%%machine learning%%' 
                OR name ILIKE '%%машинн%%обучен%%')
                AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
                AND key_skills ILIKE '%%python%%'
            GROUP BY experience
         '''
df = pd.read_sql_query(query, engine)
df

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,150474.0
2,От 3 до 6 лет,230636.0


Можно провести дополнительно исследование зарплат в регионах

In [40]:
query = f'''SELECT a.name AS area, COUNT(*) AS num_vacancies, ROUND(AVG(
            CASE
               WHEN v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL THEN (v.salary_from + v.salary_to) / 2
               WHEN v.salary_from IS NOT NULL THEN v.salary_from
               WHEN v.salary_to IS NOT NULL THEN v.salary_to
            END)) AS avg_salary
            FROM vacancies v
            JOIN areas a ON v.area_id = a.id
            WHERE (v.name ILIKE '%%data scientist%%' 
               OR v.name ILIKE '%%data science%%' 
               OR v.name ILIKE '%%исследователь данных%%' 
               OR (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%') 
               OR v.name ILIKE '%%machine learning%%' 
               OR v.name ILIKE '%%машинн%%обучен%%')
               AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
            GROUP BY a.name
            ORDER BY num_vacancies DESC, avg_salary DESC;
         '''
df = pd.read_sql_query(query, engine)
df

,area,num_vacancies,avg_salary
0,Москва,27,207630.0
1,Санкт-Петербург,7,173335.0
2,Армения,3,268863.0
3,Новосибирск,3,166333.0
4,Рязань,3,136667.0
5,Екатеринбург,3,124000.0
6,Алматы,2,94587.0
7,Владивосток,2,85000.0
8,Пермь,2,75000.0
9,Кипр,1,300000.0


В большинстве вакансий не указана заработная плата, поэтому хороший анализ зарплат сделать не получится

### Финал

Исходя из проведенного анализа, можно сделать следующие выводы:

Распределение вакансий по регионам: Большинство вакансий сосредоточено в крупных городах, таких как Москва, Санкт-Петербург, Новосибирск и других. Это может быть связано с тем, что большинство компаний-работодателей базируются в этих городах.

Работодатели: Яндекс является компанией с наибольшим количеством вакансий. Это говорит о том, что Яндекс активно нанимает новых сотрудников.

Зарплаты: Средняя зарплата для вакансий “Data Scientist” в составляет около 61900-219500.

Требуемый опыт: Количество вакансий для начинающих специалистов по данным составляет всего около 51 из 1771, большинство вакансий требуют от 1 до 3 лет опыта работы.

Сферы деятельности: Большинство работодателей указывают от 1 до 4 сфер деятельности. Это связано с тем, что компании работают в нескольких областях.

In [32]:
engine.dispose()